# iCZ870碳源氮源扫描

In [1]:
"""
碳源和氮源扫描分析脚本（三模型比较版 - 支持多交换反应）
从Excel文件读取碳源和氮源数据，测试代谢模型生长能力
支持同时比较3个模型，支持同时关闭多个交换反应
"""

import pandas as pd
import cobra
from cobra import Reaction, Configuration
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
import warnings
import sys
import os
from datetime import datetime

warnings.filterwarnings('ignore')


def setup_solver(model):
    """
    设置最佳可用求解器

    参数:
        model: COBRA模型对象

    返回:
        solver_name: 使用的求解器名称
    """
    # 求解器优先级列表
    solver_priority = ['gurobi', 'cplex', 'glpk']

    for solver in solver_priority:
        try:
            model.solver = solver
            # 测试求解器是否可用
            test_solution = model.optimize()
            print(f"✓ 使用求解器: {solver}")
            return solver
        except Exception as e:
            continue

    # 如果所有求解器都失败，使用默认求解器
    print(f"⚠️ 使用默认求解器")
    return "default"


def rename_problematic_reactions(model):
    """
    重命名与LP格式关键字冲突的反应ID

    参数:
        model: COBRA模型对象
    """
    # LP格式关键字列表
    lp_keywords = ['St', 'MAX', 'MIN', 'BOUNDS', 'BINARY', 'GENERAL',
                   'END', 'SUBJECT', 'TO', 'SOS', 'SEMI']

    renamed_count = 0
    for rxn in model.reactions:
        # 检查反应ID是否与LP关键字冲突
        rxn_id_upper = rxn.id.upper()
        if rxn_id_upper in [kw.upper() for kw in lp_keywords]:
            old_id = rxn.id
            new_id = f"{rxn.id}_rxn"

            try:
                rxn.id = new_id
                renamed_count += 1
                print(f"  ⚠️ 重命名反应: {old_id} → {new_id}")
            except Exception as e:
                print(f"  ✗ 无法重命名反应 {old_id}: {e}")

    if renamed_count > 0:
        print(f"✓ 共重命名 {renamed_count} 个有冲突的反应")

    return renamed_count


def read_source_data(excel_file):
    """
    从Excel文件读取碳源和氮源数据

    参数:
        excel_file: Excel文件路径

    返回:
        carbon_sources: 碳源列表 [(名称, ModelSEED_ID, BiGG_ID, 预期生长), ...]
        nitrogen_sources: 氮源列表
    """
    print(f"读取数据文件: {excel_file}")

    if not os.path.exists(excel_file):
        print(f"✗ 文件不存在: {excel_file}")
        print(f"当前工作目录: {os.getcwd()}")
        print(f"请确保数据文件存在")
        sys.exit(1)

    try:
        # 尝试读取Excel文件的第一个sheet
        df = pd.read_excel(excel_file, sheet_name=0, header=None)

        print(f"✓ 成功读取文件,共 {len(df)} 行")
        print(f"前5行数据预览:")
        print(df.head())
        print()

        # 查找"Carbon source"和"Nitrogen source"的行索引
        carbon_start = None
        nitrogen_start = None

        for idx, row in df.iterrows():
            if pd.notna(row.iloc[0]):
                first_col = str(row.iloc[0]).strip()
                if "carbon" in first_col.lower() and "source" in first_col.lower():
                    carbon_start = idx
                    print(f"✓ 找到碳源标题，位于第 {idx+1} 行")
                elif "nitrogen" in first_col.lower() and "source" in first_col.lower():
                    nitrogen_start = idx
                    print(f"✓ 找到氮源标题，位于第 {idx+1} 行")

        if carbon_start is None:
            print("✗ 未找到'Carbon source'标题")
            print("Excel文件的第一列应该包含'Carbon source'标题行")
            sys.exit(1)

        # 提取碳源数据
        if nitrogen_start is not None:
            carbon_df = df.iloc[carbon_start+1:nitrogen_start]
        else:
            carbon_df = df.iloc[carbon_start+1:]

        # 提取氮源数据
        if nitrogen_start is not None:
            nitrogen_df = df.iloc[nitrogen_start+1:]
        else:
            nitrogen_df = pd.DataFrame()

        # 转换为列表格式 - 碳源
        carbon_sources = []
        for idx, row in carbon_df.iterrows():
            if pd.notna(row.iloc[0]) and str(row.iloc[0]).strip():
                name = str(row.iloc[0]).strip()

                # 跳过空行或标题行
                if name.lower() in ['carbon source', 'nitrogen source', '']:
                    continue

                modelseed_id = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else ""
                bigg_id = str(row.iloc[2]).strip() if len(row) > 2 and pd.notna(row.iloc[2]) else ""

                # 处理Growth列
                if len(row) > 3 and pd.notna(row.iloc[3]):
                    growth_str = str(row.iloc[3]).strip().upper()
                    growth = growth_str == "TRUE"
                else:
                    growth = True

                if name and modelseed_id:
                    carbon_sources.append((name, modelseed_id, bigg_id, growth))

        # 转换为列表格式 - 氮源
        nitrogen_sources = []
        for idx, row in nitrogen_df.iterrows():
            if pd.notna(row.iloc[0]) and str(row.iloc[0]).strip():
                name = str(row.iloc[0]).strip()

                # 跳过空行或标题行
                if name.lower() in ['carbon source', 'nitrogen source', '']:
                    continue

                modelseed_id = str(row.iloc[1]).strip() if pd.notna(row.iloc[1]) else ""
                bigg_id = str(row.iloc[2]).strip() if len(row) > 2 and pd.notna(row.iloc[2]) else ""

                # 处理Growth列
                if len(row) > 3 and pd.notna(row.iloc[3]):
                    growth_str = str(row.iloc[3]).strip().upper()
                    growth = growth_str == "TRUE"
                else:
                    growth = True

                if name and modelseed_id:
                    nitrogen_sources.append((name, modelseed_id, bigg_id, growth))

        print(f"✓ 读取到 {len(carbon_sources)} 个碳源")
        if len(carbon_sources) > 0:
            print(f"  示例: {carbon_sources[0][0]} - {carbon_sources[0][1]}")

        print(f"✓ 读取到 {len(nitrogen_sources)} 个氮源")
        if len(nitrogen_sources) > 0:
            print(f"  示例: {nitrogen_sources[0][0]} - {nitrogen_sources[0][1]}")

        return carbon_sources, nitrogen_sources

    except Exception as e:
        print(f"✗ 读取Excel文件失败: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)


def test_source(model, source_id, exchanges_to_close, biomass_rxn, model_name="Model", source_type="carbon"):
    """
    测试单个碳源或氮源对模型生长的影响

    ✨ 新功能：支持关闭多个交换反应

    参数:
        model: COBRA模型对象
        source_id: 代谢物ID
        exchanges_to_close: 需要关闭的交换反应ID（可以是字符串或列表）
        biomass_rxn: 生物量反应ID
        model_name: 模型名称（用于日志输出）
        source_type: 源类型 ("carbon" 或 "nitrogen")

    返回:
        bool: True表示可以生长，False表示不能生长
    """
    test_model = model.copy()

    try:
        # 🌟 支持单个或多个交换反应
        # 将输入统一转换为列表格式
        if isinstance(exchanges_to_close, str):
            exchanges_list = [exchanges_to_close]
        else:
            exchanges_list = list(exchanges_to_close)

        # 1. 关闭所有指定的交换反应
        closed_count = 0
        for exchange_id in exchanges_list:
            if exchange_id in test_model.reactions:
                rxn_close = test_model.reactions.get_by_id(exchange_id)
                rxn_close.lower_bound = 0
                rxn_close.upper_bound = 0
                closed_count += 1
            else:
                print(f"  ⚠️ {model_name}: 未找到交换反应 {exchange_id}")

        if closed_count > 0:
            print(f"  {model_name}: 已关闭 {closed_count} 个交换反应")

        # 2. 检查代谢物是否存在
        if source_id not in test_model.metabolites:
            print(f"  ⚠️ {model_name}: 无代谢物 {source_id}，跳过测试")
            return False

        # 3. 创建输入反应
        source_input = Reaction(f"{source_type}_input_{source_id}")
        source_input.name = f"{source_type} source input for {source_id}"
        source_input.lower_bound = 0
        source_input.upper_bound = 10
        metabolite = test_model.metabolites.get_by_id(source_id)
        source_input.add_metabolites({metabolite: 1.0})
        test_model.add_reactions([source_input])

        # 4. 检查并设置生物量反应
        if biomass_rxn not in test_model.reactions:
            print(f"  ⚠️ {model_name}: 无生物量反应 {biomass_rxn}，跳过测试")
            return False

        biomass_reaction = test_model.reactions.get_by_id(biomass_rxn)
        test_model.objective = biomass_reaction

        # 5. 执行优化
        with test_model:
            solution = test_model.optimize()

            # 检查优化结果
            if solution is None or solution.status != 'optimal':
                return False

            growth_rate = solution.objective_value if solution.objective_value is not None else 0.0

            # 判断是否有有效生长
            if growth_rate > 1e-6:
                return True
            else:
                return False

    except Exception as e:
        # 捕获所有异常，返回False
        print(f"  ⚠️ {model_name}: 测试失败 - {type(e).__name__}: {str(e)[:100]}")
        return False


def format_worksheet(worksheet, num_rows, first_column_name):
    """
    格式化Excel工作表（3模型版本）
    """
    worksheet['A1'] = first_column_name

    # 设置列宽
    worksheet.column_dimensions['A'].width = 25
    worksheet.column_dimensions['B'].width = 18
    worksheet.column_dimensions['C'].width = 18
    worksheet.column_dimensions['D'].width = 12
    worksheet.column_dimensions['E'].width = 12
    worksheet.column_dimensions['F'].width = 18
    worksheet.column_dimensions['G'].width = 18
    worksheet.column_dimensions['H'].width = 18  # 新增第三个模型列

    # 定义样式
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=11)
    header_alignment = Alignment(horizontal='center', vertical='center')

    true_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
    true_font = Font(color="006100", bold=True)

    false_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
    false_font = Font(color="9C0006", bold=True)

    center_alignment = Alignment(horizontal='center', vertical='center')

    thin_border = Border(
        left=Side(style='thin', color='D0D0D0'),
        right=Side(style='thin', color='D0D0D0'),
        top=Side(style='thin', color='D0D0D0'),
        bottom=Side(style='thin', color='D0D0D0')
    )

    # 格式化表头
    for cell in worksheet[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment
        cell.border = thin_border

    # 格式化数据行
    for row in worksheet.iter_rows(min_row=2, max_row=num_rows+1):
        for cell in row:
            cell.alignment = center_alignment
            cell.border = thin_border

            if cell.value == "TRUE":
                cell.fill = true_fill
                cell.font = true_font
            elif cell.value == "FALSE":
                cell.fill = false_fill
                cell.font = false_font


def run_screening(data_file,
                  model1_path, model2_path, model3_path,
                  output_path,
                  model1_carbon_exchange, model1_nitrogen_exchange, model1_biomass, model1_name,
                  model2_carbon_exchange, model2_nitrogen_exchange, model2_biomass, model2_name,
                  model3_carbon_exchange, model3_nitrogen_exchange, model3_biomass, model3_name,
                  model3_id_type="modelseed"):
    """
    执行碳源和氮源扫描（三模型版本）

    参数:
        data_file: 包含碳源和氮源数据的Excel文件
        model1_path: 模型1路径
        model2_path: 模型2路径
        model3_path: 模型3路径
        output_path: 输出文件路径
        model1_carbon_exchange: 模型1的碳源交换反应ID（字符串或列表）
        model1_nitrogen_exchange: 模型1的氮源交换反应ID（字符串或列表）
        model1_biomass: 模型1的生物量反应ID
        model1_name: 模型1显示名称
        model2_carbon_exchange: 模型2的碳源交换反应ID（字符串或列表）
        model2_nitrogen_exchange: 模型2的氮源交换反应ID（字符串或列表）
        model2_biomass: 模型2的生物量反应ID
        model2_name: 模型2显示名称
        model3_carbon_exchange: 模型3的碳源交换反应ID（字符串或列表）
        model3_nitrogen_exchange: 模型3的氮源交换反应ID（字符串或列表）
        model3_biomass: 模型3的生物量反应ID
        model3_name: 模型3显示名称
        model3_id_type: 模型3使用的ID类型 ("modelseed" 或 "bigg")
    """
    print("=" * 80)
    print("碳源和氮源扫描分析程序（三模型比较版）")
    print("✨ 支持多交换反应关闭")
    print("=" * 80)
    print()

    # 读取数据
    carbon_sources, nitrogen_sources = read_source_data(data_file)
    print()

    # 加载模型1
    print("加载模型1...")
    try:
        model1 = cobra.io.read_sbml_model(model1_path)
        print(f"✓ 成功加载模型1: {model1.id}")
        print(f"  反应数: {len(model1.reactions)}, 代谢物数: {len(model1.metabolites)}")
        print("\n检查模型1的反应ID...")
        rename_problematic_reactions(model1)
        print("\n配置模型1求解器...")
        setup_solver(model1)
    except Exception as e:
        print(f"✗ 加载模型1失败: {e}")
        import traceback
        traceback.print_exc()
        return

    # 加载模型2
    try:
        model2 = cobra.io.read_sbml_model(model2_path)
        print(f"\n✓ 成功加载模型2: {model2.id}")
        print(f"  反应数: {len(model2.reactions)}, 代谢物数: {len(model2.metabolites)}")
        print("\n检查模型2的反应ID...")
        rename_problematic_reactions(model2)
        print("\n配置模型2求解器...")
        setup_solver(model2)
    except Exception as e:
        print(f"✗ 加载模型2失败: {e}")
        import traceback
        traceback.print_exc()
        return

    # 加载模型3
    try:
        model3 = cobra.io.read_sbml_model(model3_path)
        print(f"\n✓ 成功加载模型3: {model3.id}")
        print(f"  反应数: {len(model3.reactions)}, 代谢物数: {len(model3.metabolites)}")
        print("\n检查模型3的反应ID...")
        rename_problematic_reactions(model3)
        print("\n配置模型3求解器...")
        setup_solver(model3)
    except Exception as e:
        print(f"✗ 加载模型3失败: {e}")
        import traceback
        traceback.print_exc()
        return

    # 显示配置信息
    print(f"\n{'='*80}")
    print("配置信息")
    print(f"{'='*80}")
    print(f"模型1 ({model1_name}):")
    print(f"  碳源交换反应: {model1_carbon_exchange}")
    print(f"  氮源交换反应: {model1_nitrogen_exchange}")
    print(f"  生物量反应: {model1_biomass}")
    print(f"\n模型2 ({model2_name}):")
    print(f"  碳源交换反应: {model2_carbon_exchange}")
    print(f"  氮源交换反应: {model2_nitrogen_exchange}")
    print(f"  生物量反应: {model2_biomass}")
    print(f"\n模型3 ({model3_name}):")
    print(f"  碳源交换反应: {model3_carbon_exchange}")
    print(f"  氮源交换反应: {model3_nitrogen_exchange}")
    print(f"  生物量反应: {model3_biomass}")
    print(f"  ID类型: {model3_id_type}")
    print()

    # ========== 碳源扫描 ==========
    print(f"\n{'='*80}")
    print(f"开始碳源扫描，共{len(carbon_sources)}个碳源...")
    print(f"{'='*80}\n")

    carbon_results = []

    for idx, (name, modelseed_id, bigg_id, expected_growth) in enumerate(carbon_sources, 1):
        print(f"[{idx}/{len(carbon_sources)}] 测试碳源: {name} (ModelSEED: {modelseed_id}, BiGG: {bigg_id})")

        # 测试模型1（使用ModelSEED ID）
        growth_model1 = test_source(
            model=model1,
            source_id=modelseed_id,
            exchanges_to_close=model1_carbon_exchange,
            biomass_rxn=model1_biomass,
            model_name=model1_name,
            source_type="carbon"
        )

        # 测试模型2（使用BiGG ID）
        growth_model2 = False
        if bigg_id:
            growth_model2 = test_source(
                model=model2,
                source_id=bigg_id,
                exchanges_to_close=model2_carbon_exchange,
                biomass_rxn=model2_biomass,
                model_name=model2_name,
                source_type="carbon"
            )

        # 测试模型3（根据配置使用ModelSEED或BiGG ID）
        growth_model3 = False
        model3_source_id = modelseed_id if model3_id_type == "modelseed" else bigg_id
        if model3_source_id:
            growth_model3 = test_source(
                model=model3,
                source_id=model3_source_id,
                exchanges_to_close=model3_carbon_exchange,
                biomass_rxn=model3_biomass,
                model_name=model3_name,
                source_type="carbon"
            )

        carbon_results.append({
            "Source": name,
            "ModelSEED id": modelseed_id,
            "Bigg id": bigg_id if bigg_id else "",
            "Growth": "TRUE" if expected_growth else "FALSE",
            "Refer": "",
            f"Sim in {model1_name}": "TRUE" if growth_model1 else "FALSE",
            f"Sim in {model2_name}": "TRUE" if growth_model2 else "FALSE" if bigg_id else "N/A",
            f"Sim in {model3_name}": "TRUE" if growth_model3 else "FALSE" if model3_source_id else "N/A"
        })

        print(f"  结果: {model1_name}={'✓生长' if growth_model1 else '✗不生长':<8} | "
              f"{model2_name}={'✓生长' if growth_model2 else '✗不生长' if bigg_id else 'N/A':<8} | "
              f"{model3_name}={'✓生长' if growth_model3 else '✗不生长' if model3_source_id else 'N/A'}\n")

    # ========== 氮源扫描 ==========
    if nitrogen_sources:
        print(f"\n{'='*80}")
        print(f"开始氮源扫描，共{len(nitrogen_sources)}个氮源...")
        print(f"{'='*80}\n")

        nitrogen_results = []

        for idx, (name, modelseed_id, bigg_id, expected_growth) in enumerate(nitrogen_sources, 1):
            print(f"[{idx}/{len(nitrogen_sources)}] 测试氮源: {name} (ModelSEED: {modelseed_id}, BiGG: {bigg_id})")

            # 测试模型1（使用ModelSEED ID）
            growth_model1 = test_source(
                model=model1,
                source_id=modelseed_id,
                exchanges_to_close=model1_nitrogen_exchange,
                biomass_rxn=model1_biomass,
                model_name=model1_name,
                source_type="nitrogen"
            )

            # 测试模型2（使用BiGG ID）
            growth_model2 = False
            if bigg_id:
                growth_model2 = test_source(
                    model=model2,
                    source_id=bigg_id,
                    exchanges_to_close=model2_nitrogen_exchange,
                    biomass_rxn=model2_biomass,
                    model_name=model2_name,
                    source_type="nitrogen"
                )

            # 测试模型3（根据配置使用ModelSEED或BiGG ID）
            growth_model3 = False
            model3_source_id = modelseed_id if model3_id_type == "modelseed" else bigg_id
            if model3_source_id:
                growth_model3 = test_source(
                    model=model3,
                    source_id=model3_source_id,
                    exchanges_to_close=model3_nitrogen_exchange,
                    biomass_rxn=model3_biomass,
                    model_name=model3_name,
                    source_type="nitrogen"
                )

            nitrogen_results.append({
                "Source": name,
                "ModelSEED id": modelseed_id,
                "Bigg id": bigg_id if bigg_id else "",
                "Growth": "TRUE" if expected_growth else "FALSE",
                "Refer": "",
                f"Sim in {model1_name}": "TRUE" if growth_model1 else "FALSE",
                f"Sim in {model2_name}": "TRUE" if growth_model2 else "FALSE" if bigg_id else "N/A",
                f"Sim in {model3_name}": "TRUE" if growth_model3 else "FALSE" if model3_source_id else "N/A"
            })

            print(f"  结果: {model1_name}={'✓生长' if growth_model1 else '✗不生长':<8} | "
                  f"{model2_name}={'✓生长' if growth_model2 else '✗不生长' if bigg_id else 'N/A':<8} | "
                  f"{model3_name}={'✓生长' if growth_model3 else '✗不生长' if model3_source_id else 'N/A'}\n")
    else:
        nitrogen_results = []

    # ========== 生成Excel文件 ==========
    print(f"\n{'='*80}")
    print("生成结果文件...")
    print(f"{'='*80}\n")

    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # 碳源工作表
        df_carbon = pd.DataFrame(carbon_results)
        df_carbon.to_excel(writer, sheet_name='Carbon Screening', index=False)

        # 氮源工作表
        if nitrogen_results:
            df_nitrogen = pd.DataFrame(nitrogen_results)
            df_nitrogen.to_excel(writer, sheet_name='Nitrogen Screening', index=False)

        workbook = writer.book

        # 格式化碳源工作表
        format_worksheet(writer.sheets['Carbon Screening'], len(carbon_results), "Carbon source")

        # 格式化氮源工作表
        if nitrogen_results:
            format_worksheet(writer.sheets['Nitrogen Screening'], len(nitrogen_results), "Nitrogen source")

    print(f"✓ 完成！结果已保存到: {output_path}\n")

    # 打印统计信息
    print(f"{'='*80}")
    print("统计信息")
    print(f"{'='*80}")
    print("\n【碳源扫描结果】")
    carbon_growth_1 = sum(1 for r in carbon_results if r[f"Sim in {model1_name}"] == "TRUE")
    carbon_growth_2 = sum(1 for r in carbon_results if r[f"Sim in {model2_name}"] == "TRUE")
    carbon_growth_3 = sum(1 for r in carbon_results if r[f"Sim in {model3_name}"] == "TRUE")
    carbon_total = len(carbon_results)
    print(f"  {model1_name}模型: {carbon_growth_1:2d}/{carbon_total} 个碳源支持生长 ({carbon_growth_1/carbon_total*100:.1f}%)")
    print(f"  {model2_name}模型: {carbon_growth_2:2d}/{carbon_total} 个碳源支持生长 ({carbon_growth_2/carbon_total*100:.1f}%)")
    print(f"  {model3_name}模型: {carbon_growth_3:2d}/{carbon_total} 个碳源支持生长 ({carbon_growth_3/carbon_total*100:.1f}%)")

    if nitrogen_results:
        print("\n【氮源扫描结果】")
        nitrogen_growth_1 = sum(1 for r in nitrogen_results if r[f"Sim in {model1_name}"] == "TRUE")
        nitrogen_growth_2 = sum(1 for r in nitrogen_results if r[f"Sim in {model2_name}"] == "TRUE")
        nitrogen_growth_3 = sum(1 for r in nitrogen_results if r[f"Sim in {model3_name}"] == "TRUE")
        nitrogen_total = len(nitrogen_results)
        print(f"  {model1_name}模型: {nitrogen_growth_1:2d}/{nitrogen_total} 个氮源支持生长 ({nitrogen_growth_1/nitrogen_total*100:.1f}%)")
        print(f"  {model2_name}模型: {nitrogen_growth_2:2d}/{nitrogen_total} 个氮源支持生长 ({nitrogen_growth_2/nitrogen_total*100:.1f}%)")
        print(f"  {model3_name}模型: {nitrogen_growth_3:2d}/{nitrogen_total} 个氮源支持生长 ({nitrogen_growth_3/nitrogen_total*100:.1f}%)")

    print(f"\n{'='*80}\n")


if __name__ == "__main__":
    # ========== 配置参数 ==========

    # 数据文件（包含碳源和氮源信息的Excel文件）
    DATA_FILE = "carbon_nitrogen_data.xlsx"

    # 模型文件路径
    MODEL1_PATH = "iCZ870_MM.xml"
    MODEL2_PATH = "other model/iCGB21FR.xml"
    MODEL3_PATH = "other model/iCW773R.xml"

    # 输出文件路径
    OUTPUT_PATH = "screening_results_3models.xlsx"

    # 模型1配置 - 使用ModelSEED ID
    MODEL1_NAME = "iCZ870"
    MODEL1_CARBON_EXCHANGE = "EX_cpd00027_e"      # 葡萄糖交换反应
    MODEL1_NITROGEN_EXCHANGE = "EX_cpd00013_e"    # NH4+交换反应
    MODEL1_BIOMASS = "Bio_cgATCC14067_c"          # 生物量反应

    # 模型2配置 - 使用BiGG ID
    MODEL2_NAME = "iCGB21FR"
    MODEL2_CARBON_EXCHANGE = "EX_glc__D_e"        # 葡萄糖交换反应
    MODEL2_NITROGEN_EXCHANGE = ["EX_nh4_e", "EX_urea_e"]  # NH4+交换反应（可以是列表）
    MODEL2_BIOMASS = "Growth"                     # 生物量反应

    # 模型3配置 -
    MODEL3_NAME = "iCW773R"
    MODEL3_CARBON_EXCHANGE = "EX_glc_e_reverse"
    MODEL3_NITROGEN_EXCHANGE = "EX_nh4_e_reverse"
    MODEL3_BIOMASS = "CG_biomass_cgl_ATCC13032"
    MODEL3_ID_TYPE = "bigg"

    # ========== 运行分析 ==========

    run_screening(
        data_file=DATA_FILE,
        model1_path=MODEL1_PATH,
        model2_path=MODEL2_PATH,
        model3_path=MODEL3_PATH,
        output_path=OUTPUT_PATH,
        model1_carbon_exchange=MODEL1_CARBON_EXCHANGE,
        model1_nitrogen_exchange=MODEL1_NITROGEN_EXCHANGE,
        model1_biomass=MODEL1_BIOMASS,
        model1_name=MODEL1_NAME,
        model2_carbon_exchange=MODEL2_CARBON_EXCHANGE,
        model2_nitrogen_exchange=MODEL2_NITROGEN_EXCHANGE,
        model2_biomass=MODEL2_BIOMASS,
        model2_name=MODEL2_NAME,
        model3_carbon_exchange=MODEL3_CARBON_EXCHANGE,
        model3_nitrogen_exchange=MODEL3_NITROGEN_EXCHANGE,
        model3_biomass=MODEL3_BIOMASS,
        model3_name=MODEL3_NAME,
        model3_id_type=MODEL3_ID_TYPE
    )

碳源和氮源扫描分析程序（三模型比较版）
✨ 支持多交换反应关闭

读取数据文件: carbon_nitrogen_data.xlsx
✓ 成功读取文件,共 58 行
前5行数据预览:
               0             1         2       3  \
0  Carbon source  Modelseed id   Bigg id  Growth   
1        Glucose    cpd00027_e  glc__D_c    True   
2       Fructose    cpd00082_e     fru_c    True   
3        Sucrose    cpd00076_e    sucr_e    True   
4      Trehalose    cpd00794_e     tre_e    True   

                                                   4  
0                                              Refer  
1  Cocaign, M., Monnet, C. and Lindley, N.D., 199...  
2  Dominguez, H., Rollin, C., Guyonvarch, A., Gue...  
3  Dominguez, H. and Lindley, N.D., 1996. Complet...  
4  Kotrba, P., Inui, M. and Yukawa, H., 2001. The...  

✓ 找到碳源标题，位于第 1 行
✓ 找到氮源标题，位于第 44 行
✓ 读取到 42 个碳源
  示例: Glucose - cpd00027_e
✓ 读取到 14 个氮源
  示例: Urea - cpd00073_e

加载模型1...
Set parameter Username
Set parameter LicenseID to value 2723056
Academic license - for non-commercial use only - expires 2026-10-16


No objective coefficients in model. Unclear what should be optimized


✓ 成功加载模型1: GEM_model
  反应数: 1752, 代谢物数: 1660

检查模型1的反应ID...

配置模型1求解器...
✓ 使用求解器: gurobi


SBML package 'layout' not supported by cobrapy, information is not parsed
https://juser.fz-juelich.de/record/188973 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id



✓ 成功加载模型2: iCBG21FR
  反应数: 1539, 代谢物数: 1042

检查模型2的反应ID...
  ⚠️ 重命名反应: St → St_rxn
✓ 共重命名 1 个有冲突的反应

配置模型2求解器...
✓ 使用求解器: gurobi


'' is not a valid SBML 'SId'.



✓ 成功加载模型3: 
  反应数: 1850, 代谢物数: 949

检查模型3的反应ID...

配置模型3求解器...
✓ 使用求解器: gurobi

配置信息
模型1 (iCZ870):
  碳源交换反应: EX_cpd00027_e
  氮源交换反应: EX_cpd00013_e
  生物量反应: Bio_cgATCC14067_c

模型2 (iCGB21FR):
  碳源交换反应: EX_glc__D_e
  氮源交换反应: ['EX_nh4_e', 'EX_urea_e']
  生物量反应: Growth

模型3 (iCW773R):
  碳源交换反应: EX_glc_e_reverse
  氮源交换反应: EX_nh4_e_reverse
  生物量反应: CG_biomass_cgl_ATCC13032
  ID类型: bigg


开始碳源扫描，共42个碳源...

[1/42] 测试碳源: Glucose (ModelSEED: cpd00027_e, BiGG: glc__D_c)
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpn15z17v2.lp
Reading time = 0.01 seconds
: 1660 rows, 3504 columns, 15520 nonzeros
  iCZ870: 已关闭 1 个交换反应
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpmqjwnq78.lp
Reading time = 0.01 seconds
: 1042 rows, 3078 columns, 13304 nonzeros
  iCGB21FR: 已关闭 1 个交换反应
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmp2dbg1jk2.lp
Reading time = 0.01 seconds
: 949 rows, 3700 columns, 15396 nonzeros
  iCW773R: 已关闭 1 个交换反应
  结果: iCZ870=✓生长     

In [2]:
import subprocess
cmd_str = "python ./script/model_comparison_analyzer.py screening_results_3models.xlsx"
subprocess.run(cmd_str, shell=True)

CompletedProcess(args='python ./script/model_comparison_analyzer.py screening_results_3models.xlsx', returncode=0)